In [1]:
import numpy as np
import torch
import sys
import os
import time
import xlwt
import csv
import random
import math
import copy
import pickle
import gc

# masterPath = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.insert(1, os.path.join(masterPath, 'centralRepo'))
sys.path.insert(1, '../centralRepo')
from eegDataset import eegDataset
from baseModel import baseModel
from networks import *
import transforms
from saveData import fetchData
import seaborn as sns
import matplotlib.pyplot as plt
import statistics

In [2]:
config = {}
config['modelArguments'] = {'nChan': 22, 'nTime': 1000, 'dropoutP': 0.5,
                                    'nBands':9, 'm' : 32, 'temporalLayer': 'LogVarLayer',
                                    'nClass': 4, 'doWeightNorm': True}

# Training related details    
config['modelTrainArguments'] = {'stopCondi':  {'c': {'Or': {'c1': {'MaxEpoch': {'maxEpochs': 1000, 'varName' : 'epoch'}},
                                                   'c2': {'NoDecrease': {'numEpochs' : 200, 'varName': 'valInacc'}} } }},
      'classes': [0,1], 'sampler' : 'RandomSampler', 'loadBestModel': True,
      'bestVarToCheck': 'valInacc', 'continueAfterEarlystop':False,'lr': 1e-3}

config['batchSize'] = 16
nGPU = 0

In [3]:
config['inDataPath'] = '../../data/bci42a/multiviewPython'
config['inLabelPath'] = '../../data//bci42a/multiviewPython/dataLabels.csv'
fbcnet_dataset = eegDataset(dataPath = config['inDataPath'], dataLabelsPath= config['inLabelPath'], preloadData = False, transform= None)
FBCNET_ROOT_PATH = '../../output/bci42a/ses2Test/fbcnet_loso_finetune/'
fbcnet = FBCNet(**config['modelArguments'])

config['inDataPath'] = '../../data/bci42a/rawPython'
config['inLabelPath'] = '../../data//bci42a/rawPython/dataLabels.csv'
eegnet_dataset = eegDataset(dataPath = config['inDataPath'], dataLabelsPath= config['inLabelPath'], preloadData = False, transform= None)
EEGNET_ROOT_PATH = '../../output/bci42a/ses2Test/eegnet_loso_finetune/'
eegnet = eegNet(**config['modelArguments'])

config['inDataPath'] = '../../data/bci42a/rawPython'
config['inLabelPath'] = '../../data//bci42a/rawPython/dataLabels.csv'
DEEPCONVNET_ROOT_PATH = '../../output/bci42a/ses2Test/deepconvnet_loso_finetune/'
deepconvnet = deepConvNet(**config['modelArguments'])

In [4]:
def test_model(test_sub, data, model, session='1'):
        subIdxTest = [i for i, x in enumerate(data.labels) if x[3] in test_sub and x[4] == session]
        testData = copy.deepcopy(data)
        testData.createPartialDataset(subIdxTest, loadNonLoadedData = True)

        testResults, pred, act, pred_prob = model.test_preds(testData)

        del testData
        gc.collect()

        return testResults, pred, act, pred_prob

In [5]:
subs = list(range(9))
accs = []
all_preds = {}
all_pred_probs = {}
all_losses = {}
all_test_results = {}
all_acts = {}
all_train_preds = {}
all_train_pred_probs = {}
all_train_losses = {}
all_train_results = {}
models = ['fbcnet', 'eegnet', 'deepconvnet']
datasets = [fbcnet_dataset, eegnet_dataset, eegnet_dataset]
networks = [fbcnet, eegnet, deepconvnet]
ROOTS = [FBCNET_ROOT_PATH, EEGNET_ROOT_PATH, DEEPCONVNET_ROOT_PATH]
# models = ['fbcnet', 'deepconvnet']
# datasets = [fbcnet_dataset, eegnet_dataset]
# networks = [fbcnet, deepconvnet]
# ROOTS = [FBCNET_ROOT_PATH, DEEPCONVNET_ROOT_PATH]
for test_sub in subs:
    test_sub_str = '00' + str(test_sub + 1)
    preds = []
    pred_probs = []
    train_preds = []
    train_pred_probs = []
    results = {}
    train_results = {}
    train_losses = []
    print("Testing subject " + test_sub_str + " with all models...")
    for i, (net, ROOT_PATH) in enumerate(zip(networks, ROOTS)):
        state_dict_path = os.path.join(ROOT_PATH, 'finetuned_sub' + test_sub_str, 'network_state_dict.pth')
        netInitState = torch.load(state_dict_path)
        net.load_state_dict(netInitState, strict=False)
        outPathSub = None
        model = baseModel(net=net, resultsSavePath=outPathSub, batchSize= config['batchSize'], nGPU = nGPU)
        testResults, pred, act, pred_prob = test_model(test_sub_str, datasets[i], model, session='0')
        train_preds.append(pred)
        train_pred_probs.append(pred_prob)
        train_results[models[i]] = testResults
        train_losses.append(testResults['loss'])

        testResults, pred, act, pred_prob = test_model(test_sub_str, datasets[i], model)
        preds.append(pred)
        pred_probs.append(pred_prob)
        results[models[i]] = testResults
        print("Results for model", models[i], testResults)
    
    correct = 0
    for i in range(len(act)):
        for pred in preds:
            if(pred[i] == act[i]):
                correct += 1
                break
    
    all_preds[test_sub] = preds
    all_pred_probs[test_sub] = pred_probs
    all_acts[test_sub] = act
    all_test_results[test_sub] = results
    
    all_train_preds[test_sub] = train_preds
    all_train_pred_probs[test_sub] = train_pred_probs
    all_train_losses[test_sub] = train_losses
    all_train_results[test_sub] = train_results
    
    acc = correct/len(act)
    accs.append(acc)
    print("Theoretical max accuracy:", acc)

Testing subject 001 with all models...
Code will be running on device  cuda:0


../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs
../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


Results for model fbcnet {'acc': 0.7638888888888888, 'cm': array([[55, 11,  2,  4],
       [ 4, 68,  0,  0],
       [ 6,  4, 32, 30],
       [ 0,  0,  7, 65]]), 'loss': 0.5886249012417264}
Code will be running on device  cuda:0


../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs
../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


Results for model eegnet {'acc': 0.7916666666666666, 'cm': array([[54, 11,  3,  4],
       [ 9, 63,  0,  0],
       [ 3,  0, 54, 15],
       [ 4,  0, 11, 57]]), 'loss': 0.5459111001756456}
Code will be running on device  cuda:0


...

../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs
../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


Results for model eegnet {'acc': 0.7083333333333334, 'cm': array([[66,  2,  3,  1],
       [ 1, 47, 23,  1],
       [ 4, 16, 44,  8],
       [ 6,  4, 15, 47]]), 'loss': 0.7501369052463107}
Code will be running on device  cuda:0


../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


Results for model deepconvnet {'acc': 0.625, 'cm': array([[68,  4,  0,  0],
       [ 4, 44, 24,  0],
       [ 9, 18, 39,  6],
       [10, 12, 21, 29]]), 'loss': 0.9070219463772244}
Theoretical max accuracy: 0.8993055555555556


../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


In [6]:
accs

[0.9583333333333334,
 0.8194444444444444,
 0.9791666666666666,
 0.9201388888888888,
 0.8854166666666666,
 0.8645833333333334,
 0.9583333333333334,
 0.9340277777777778,
 0.8993055555555556]

In [7]:
avg_acc = sum(accs) / len(accs)
print("Average theoretical max accuracy:", avg_acc)

Average theoretical max accuracy: 0.9131944444444444


In [8]:
accs.append(avg_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in accs])
print(tabstr)

95.833 & 81.944 & 97.917 & 92.014 & 88.542 & 86.458 & 95.833 & 93.403 & 89.931 & 91.319


In [9]:
voted_accs = []
for test_sub in subs:
    voted_pred = []
#     test_sub = 1
    preds = all_preds[test_sub]
    act = all_acts[test_sub]
    for i in range(len(preds[0])):
        votes = {l:0 for l in range(4)}
        for pred in preds:
            votes[pred[i]]+=1
        voted_pred.append(max(votes, key=votes.get))

    correct = 0
    for pred, act_val in zip(voted_pred, act):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(act)
    voted_accs.append(voted_acc)
    print("Voted accuracy:", voted_acc)
    
avg_voted_acc = sum(voted_accs) / len(voted_accs)
print("Average voted accuracy:", avg_voted_acc)
print("Median voted accuracy:", statistics.median(voted_accs))
voted_accs.append(avg_voted_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in voted_accs])
print(tabstr)

Voted accuracy: 0.8020833333333334
Voted accuracy: 0.5972222222222222
Voted accuracy: 0.8229166666666666
Voted accuracy: 0.7118055555555556
Voted accuracy: 0.5902777777777778
Voted accuracy: 0.6111111111111112
Voted accuracy: 0.8159722222222222
Voted accuracy: 0.8125
Voted accuracy: 0.7256944444444444
Average voted accuracy: 0.7210648148148148
Median voted accuracy: 0.7256944444444444
80.208 & 59.722 & 82.292 & 71.181 & 59.028 & 61.111 & 81.597 & 81.250 & 72.569 & 72.106


### Soft probability combine

In [10]:
comb_accs = []
for test_sub in subs:
    pred_probs = np.array(all_pred_probs[test_sub])
    n, h, w, c = pred_probs.shape
    pred_probs = np.reshape(pred_probs, (n, h*w, c))
    class_probs = np.sum(pred_probs, axis=0)
    soft_preds = np.argmax(class_probs, axis=1)
    correct = 0
    for pred, act_val in zip(soft_preds, all_acts[test_sub]):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(all_acts[test_sub])
    comb_accs.append(voted_acc)
    print("Soft probability combined accuracy:", voted_acc)

avg_comb_acc = sum(comb_accs) / len(comb_accs)
print("Average combined accuracy:", avg_comb_acc)
print("Median combined accuracy:", statistics.median(comb_accs))
comb_accs.append(avg_comb_acc)
tabstr = ' \n'.join(["{0:.3f}".format(x*100) for x in comb_accs])
print(tabstr)

Soft probability combined accuracy: 0.8263888888888888
Soft probability combined accuracy: 0.625
Soft probability combined accuracy: 0.8715277777777778
Soft probability combined accuracy: 0.7604166666666666
Soft probability combined accuracy: 0.4722222222222222
Soft probability combined accuracy: 0.6770833333333334
Soft probability combined accuracy: 0.8368055555555556
Soft probability combined accuracy: 0.8055555555555556
Soft probability combined accuracy: 0.7604166666666666
Average combined accuracy: 0.7372685185185185
Median combined accuracy: 0.7604166666666666
82.639 
62.500 
87.153 
76.042 
47.222 
67.708 
83.681 
80.556 
76.042 
73.727


In [11]:
cms = {test_sub: {train_sub: results['cm'] for train_sub, results in test_results.items()} for test_sub, test_results in all_train_results.items()}

In [12]:
metrics = {test_sub: {train_sub: np.array([cm[i][i]/np.sum(cm[i,:]) for i in range(4)]) for train_sub, cm in test_results.items()} for test_sub, test_results in cms.items()}

In [13]:
recall_sums = {test_sub: np.array([np.sum(recalls) for train_sub, recalls in test_results.items()]) for test_sub, test_results in metrics.items()}

In [14]:
recall_sums[0]

array([3.90277778, 4.        , 4.        ])

In [15]:
np.argsort(recall_sums[0])

array([0, 1, 2])

In [16]:
comb_accs = []
for test_sub in subs:
    pred_probs = np.array(all_pred_probs[test_sub])
    n, h, w, c = pred_probs.shape
    pred_probs = np.reshape(pred_probs, (n, h*w, c))
#     print(pred_probs)
    for i in range(len(pred_probs)):
#         print(pred_probs[i])
        for j in range(len(pred_probs[i])):
            pred_probs[i][j] = np.multiply(pred_probs[i][j], metrics[test_sub][models[i]])
    class_probs = np.sum(pred_probs, axis=0)
    soft_preds = np.argmax(class_probs, axis=1)
    correct = 0
    for pred, act_val in zip(soft_preds, all_acts[test_sub]):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(all_acts[test_sub])
    comb_accs.append(voted_acc)
    print("Soft probability combined accuracy:", voted_acc)

avg_comb_acc = sum(comb_accs) / len(comb_accs)
print("Average combined accuracy:", avg_comb_acc)
print("Median combined accuracy:", statistics.median(comb_accs))
comb_accs.append(avg_comb_acc)
tabstr = '\n'.join(["{0:.3f}".format(x*100) for x in comb_accs])
print(tabstr)

Soft probability combined accuracy: 0.8263888888888888
Soft probability combined accuracy: 0.625
Soft probability combined accuracy: 0.9375
Soft probability combined accuracy: 0.7604166666666666
Soft probability combined accuracy: 0.4722222222222222
Soft probability combined accuracy: 0.6736111111111112
Soft probability combined accuracy: 0.8368055555555556
Soft probability combined accuracy: 0.8055555555555556
Soft probability combined accuracy: 0.7638888888888888
Average combined accuracy: 0.7445987654320988
Median combined accuracy: 0.7638888888888888
82.639
62.500
93.750
76.042
47.222
67.361
83.681
80.556
76.389
74.460


In [17]:
voted_accs = []
for test_sub in subs:
    voted_pred = []
#     test_sub = 1
    preds = all_preds[test_sub]
    act = all_acts[test_sub]
    for i in range(len(preds[0])):
        votes = {l:0 for l in range(4)}
        for j, pred in enumerate(preds):
            votes[pred[i]]+=metrics[test_sub][models[j]][pred[i]]
        voted_pred.append(max(votes, key=votes.get))

    correct = 0
    for pred, act_val in zip(voted_pred, act):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(act)
    voted_accs.append(voted_acc)
    print("Voted accuracy:", voted_acc)
    
avg_voted_acc = sum(voted_accs) / len(voted_accs)
print("Average voted accuracy:", avg_voted_acc)
print("Median voted accuracy:", statistics.median(voted_accs))
voted_accs.append(avg_voted_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in voted_accs])
print(tabstr)

Voted accuracy: 0.8020833333333334
Voted accuracy: 0.5798611111111112
Voted accuracy: 0.8229166666666666
Voted accuracy: 0.7118055555555556
Voted accuracy: 0.5902777777777778
Voted accuracy: 0.6180555555555556
Voted accuracy: 0.8159722222222222
Voted accuracy: 0.8125
Voted accuracy: 0.7256944444444444
Average voted accuracy: 0.7199074074074074
Median voted accuracy: 0.7256944444444444
80.208 & 57.986 & 82.292 & 71.181 & 59.028 & 61.806 & 81.597 & 81.250 & 72.569 & 71.991
